In [7]:
import sys

# sys.path.append(r'/Users/laisky/repo/laisky/ramjet/ramjet/settings')
sys.path.append(r'/opt/configs/ramjet')
sys.path
import prd

In [14]:
from tweepy import API, OAuthHandler


auth = OAuthHandler(prd.CONSUMER_KEY, prd.CONSUMER_SECRET)
auth.set_access_token(prd.ACCESS_TOKEN, prd.ACCESS_TOKEN_SECRET)
api = API(auth, wait_on_rate_limit=True)
api.me()

User(_api=<tweepy.api.API object at 0x10712a518>, _json={'id': 105351466, 'id_str': '105351466', 'name': 'Laisky', 'screen_name': 'ppcelery', 'location': '♥ Shanghai', 'profile_location': None, 'description': '热爱文学的理科生，本质是一台高语速大存储的复读机。', 'url': 'https://t.co/tmLO9n4B1E', 'entities': {'url': {'urls': [{'url': 'https://t.co/tmLO9n4B1E', 'expanded_url': 'https://about.me/laisky', 'display_url': 'about.me/laisky', 'indices': [0, 23]}]}, 'description': {'urls': []}}, 'protected': False, 'followers_count': 7492, 'friends_count': 227, 'listed_count': 132, 'created_at': 'Sat Jan 16 03:02:39 +0000 2010', 'favourites_count': 838, 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'verified': False, 'statuses_count': 23799, 'lang': None, 'status': {'created_at': 'Fri Jun 21 05:22:38 +0000 2019', 'id': 1141939461363654656, 'id_str': '1141939461363654656', 'text': '你们有些人说话真脏，受过教育吗？mute 都 mute 不过来。', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls'

In [42]:
api.user_timeline(count=10)

[Status(_api=<tweepy.api.API object at 0x10712a518>, _json={'created_at': 'Fri Jun 21 06:23:44 +0000 2019', 'id': 1141954835958259712, 'id_str': '1141954835958259712', 'text': '市里是简单的粗暴的订了一个罚款和征信规则，然后把压力全部扔给街道，街道目前的做法是要求楼组长和党员起带头作用，主动轮班做志愿者（就是问你是什么垃圾的那些阿姨）。个别富裕区会提供一些现金奖励，但是哪一个看上去都不像是长久之策，比如已经有人去市长热线举报志愿者不作为，不帮他分捡垃圾了', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': 1141934448256008192, 'in_reply_to_status_id_str': '1141934448256008192', 'in_reply_to_user_id': 105351466, 'in_reply_to_user_id_str': '105351466', 'in_reply_to_screen_name': 'ppcelery', 'user': {'id': 105351466, 'id_str': '105351466', 'name': 'Laisky', 'screen_name': 'ppcelery', 'location': '♥ Shanghai', 'description': '热爱文学的理科生，本质是一台高语速大存储的复读机。', 'url': 'https://t.co/tmLO9n4B1E', 'entities': {'url': {'urls': [{'url': 'https://t.co/tmLO9n4B1E', 'expanded_ur

In [57]:
# delete tweets

from datetime import datetime
import traceback

from pymongo import MongoClient


mongo = MongoClient("localhost", 27017)
tweets = mongo['twitter']['tweets']


def is_status_tobe_delete(tweet):
    if len(tweet.get('entities', {}).get("hashtags", [])) >= 1:
        return False
    
    if tweet['created_at'] < datetime(2017,6,5):
        return True

def delete():
#     last_tweets = api.user_timeline(count=1)
#     if not last_tweets:
#         return
    
#     print(last_tweets[0].created_at < datetime(2019,7,1))
#     print(last_tweets[0])
#     print(dir(last_tweets[0]))
#     return

#     current_id = last_tweets[0].id
    current_id = "379207474345046017"
    n = n_del = 1
#         tweets = api.user_timeline(max_id=current_id, count=1000)
    for tweet in tweets.find({'user.screen_name': 'ppcelery'}).sort("_id", -1):
        n += 1

        if is_status_tobe_delete(tweet):
            n_del += 1
            print("delete", tweet['id'],tweet['created_at'])
            try:
                api.destroy_status(tweet['id'])
            except Exception:
                traceback.print_exc()
        else:
            current_id = tweet['id']

        if n % 1000 == 0:
            print(">> scan {} statuses".format(n-1))
        if n_del % 100 == 0:
            print(">> delete {} statuses".format(n_del-1))


delete()

>> scan 999 statuses
>> scan 1999 statuses
>> scan 2999 statuses
>> scan 3999 statuses
>> scan 4999 statuses
>> scan 5999 statuses
>> scan 6999 statuses
>> scan 7999 statuses
>> scan 8999 statuses
>> scan 9999 statuses
>> scan 10999 statuses
>> scan 11999 statuses
>> scan 12999 statuses
>> scan 13999 statuses
>> scan 14999 statuses
>> scan 15999 statuses
>> scan 16999 statuses
>> scan 17999 statuses
>> scan 18999 statuses
>> scan 19999 statuses
>> scan 20999 statuses
>> scan 21999 statuses
>> scan 22999 statuses
>> scan 23999 statuses
>> scan 24999 statuses
>> scan 25999 statuses
>> scan 26999 statuses
>> scan 27999 statuses


KeyboardInterrupt: 

In [64]:
from pathlib import Path


p = Path('/a', 'v', 'c')
open(p)

FileNotFoundError: [Errno 2] No such file or directory: '/a/v/c'

In [67]:
# download image
import requests
from pathlib import Path

dirpath = r'/var/www/uploads/twitter'
# dirpath = r'/Users/laisky/Downloads'


def download_images_for_tweet(tweet):
    for img in tweet['entities']['media']:
        with requests.get(img['media_url_https']+":orig") as r:
            if r.status_code != 200:
                print("got error", r.text)
                continue
                
            fpath = Path(dirpath, img['media_url_https'].split("/")[-1])
            if fpath.is_file():
                continue
                
            with open(fpath, 'wb') as f:
                f.write(r.content)    
            print("tweet img ok", tweet['id'], fpath)
            

def download_images():
    for tweet in tweets.find({"entities.media": {"$exists": 1}}).sort("_id", -1):
        download_images_for_tweet(tweet)
        
        
download_images()
        

tweet img ok 719103735004200961 /Users/laisky/Downloads/CfrFCRcUEAIFaEe.jpg
tweet img ok 719468584750096384 /Users/laisky/Downloads/CfwQ3VxVIAAj1Yv.jpg
got error 


UnboundLocalError: local variable 'fpath' referenced before assignment